In [1]:
!wget https://github.com/alexeygrigorev/ai-bootcamp-codespace/raw/refs/heads/main/week1/docs.py

--2025-10-25 18:19:44--  https://github.com/alexeygrigorev/ai-bootcamp-codespace/raw/refs/heads/main/week1/docs.py
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/ai-bootcamp-codespace/refs/heads/main/week1/docs.py [following]
--2025-10-25 18:19:44--  https://raw.githubusercontent.com/alexeygrigorev/ai-bootcamp-codespace/refs/heads/main/week1/docs.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583 (8.4K) [text/plain]
Saving to: ‘docs.py.3’

docs.py.3           100%[===================>]   8.38K  --.-KB/s    in 0s      

2025-10-25 18:19:45 (82.0 MB/s) - ‘docs.py.3’ saved [8

In [2]:
import docs
from minsearch import Index
from typing import Any, Dict, List, TypedDict
from typing import Optional
from toyaikit.llm import OpenAIClient
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIResponsesRunner
from toyaikit.chat.runners import DisplayingRunnerCallback
from toyaikit.tools import Tools

In [3]:
github_data = docs.read_github_data()
parsed_data = docs.parse_data(github_data)
chunks = docs.chunk_documents(parsed_data)

In [4]:
index = Index(
    text_fields=["content", "filename", "title", "description"],
)

index.fit(chunks)

In [5]:
class SearchResult(TypedDict):
    """Represents a single search result entry."""
    start: int
    content: str
    title: str
    description: str
    filename: str

In [6]:
def search(query: str) -> List[SearchResult]:
    """
    Search the index for documents matching the given query.

    Args:
        query (str): The search query string.

    Returns:
        List[SearchResult]: A list of search results. Each result dictionary contains:
            - start (int): The starting position or offset within the source file.
            - content (str): A text excerpt or snippet containing the match.
            - title (str): The title of the matched document.
            - description (str): A short description of the document.
            - filename (str): The path or name of the source file.
    """
    return index.search(
        query=query,
        num_results=5,
    )

In [7]:
file_index = {}

for doc in parsed_data:
    filename = doc['filename']
    file_index[filename] = doc

In [8]:

def read_file(filename: str) -> Optional[str]:
    """
    Retrieve the content of a file from the repository.

    Args:
        filename (str): The name or path of the file to read.

    Returns:
        Optional[str]: The file content as a string if the file exists;
        otherwise, returns None.
    """
    if filename in file_index:
        return file_index[filename]['content']
    return None

In [9]:
instructions = """
You are an assistant that helps improve and generate high-quality documentation for the project.

You have access to the following tools:
- search — Use this to explore topics in depth. Make multiple search calls if needed to gather comprehensive information.
- read_file — Use this when code snippets are missing or when you need to retrieve the full content of a file for context.

If `read_file` cannot be used or the file content is unavailable, clearly state:
> "Unable to verify with read_file."

When answering a question:
1. Provide file references for all source materials.  
   Use this format:  
   [{filename}](https://github.com/evidentlyai/docs/blob/main/{filename})
2. If the topic is covered in multiple documents, cite all relevant sources.
3. Include code examples whenever they clarify or demonstrate the concept.
4. Be concise, accurate, and helpful — focus on clarity and usability for developers.
5. If documentation is missing or unclear, infer from context and note that explicitly.

Critical Rule

Before generating or finalizing any code example or technical explanation, you must always call `read_file` to cross-check the correctness of the code.
Do not rely solely on search results or assumptions — always verify by reading the actual file content.

Example Citation:
See the full implementation in [metrics/api_reference.md](https://github.com/evidentlyai/docs/blob/main/metrics/api_reference.md).
""".strip()


In [10]:
agent_tools = Tools()

agent_tools.add_tool(search)
agent_tools.add_tool(read_file)

In [11]:
chat_interface = IPythonChatInterface()

runner = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=instructions,
    chat_interface=chat_interface,
    llm_client=OpenAIClient()
)

In [12]:
runner.run()

You: hola


You: que haces tu


KeyboardInterrupt: Interrupted by user